In [174]:
import pandas as pd

* NaN 用 0 取代
* 去除最後一筆資料 (結算最後比分)
* 加上隊伍資訊
* Action == OP 的 column 去除，reset index

In [187]:
def get_data(link, team, set_num):
    df = pd.DataFrame()
    for i in range(1, set_num+1):
        new_df = pd.read_excel(link, sheet_name = team + ' Set' + str(i))
        df = pd.concat([df, new_df])
    
    df = df.fillna(0)
    df.insert(0, column='Team', value=[team for i in range(len(df))])
    df = df.rename(columns={'No.': 'No'})
    drop_col = df.columns.tolist()
    col = ['Team', 'No', 'Space', 'Action', 'Erros', 'Attempts', 'Score']
    for c in col:
        drop_col.remove(c)
    df = df.drop(columns=drop_col)
    df = df.drop([idx for idx, row in df.iterrows() if(isinstance(row['No'], int)==False and isinstance(row['No'], float)==False)])
    df = df.drop(df.query('Space not in [1, 2, 3, 4, 5, 6, 7, 8, 9, \'M\', \'Y\', \'L\', \'FZ\']').index)
    df = df.drop(df.query('Action not in [\'A\', \'B\', \'D\', \'FS\', \'G\', \'JS\', \'FS\', \'R\']').index)

    df = df.drop(df.query('Score not in [1, 1.0, 0, 0.0]').index)
    df = df.drop(df.query('Erros not in [1, 1.0, 0, 0.0]').index)


    df = df.astype({'No': 'int32'})
    df = df.astype({'No': 'str'})
    

    df['Space'].replace({'M': 10, 'Y': 11, 'L': 12, 'FZ': 13}, inplace=True)
    
    df['Action'].replace({'A': 1, 'B': 2, 'D': 3, 'FS': 4, 'G': 5, 'JS': 6, 'OP': 7, 'R': 8}, inplace=True)

    return df

In [188]:
df_dict = {}

df_dict['df_IRI'] = get_data('Data/2022 VNL/IRI VS POL.xlsx', 'IRI', 5)
df_dict['df_POL_1'] = get_data('Data/2022 VNL/IRI VS POL.xlsx', 'POL', 5)

df_dict['df_ITA_1'] = get_data('Data/2022 VNL/ITA VS FRA.xlsx', 'ITA', 3)
df_dict['df_FRA_1'] = get_data('Data/2022 VNL/ITA VS FRA.xlsx', 'FRA', 3)

df_dict['df_ITA_2'] = get_data('Data/2022 VNL/ITA VS NED.xlsx', 'ITA', 4)
df_dict['df_NED'] = get_data('Data/2022 VNL/ITA VS NED.xlsx', 'NED', 4)

df_dict['df_ITA_3'] = get_data('Data/2022 VNL/ITA VS POL.xlsx', 'ITA', 3)
df_dict['df_POL_2'] = get_data('Data/2022 VNL/ITA VS POL.xlsx', 'POL', 3)

df_dict['df_JAP'] = get_data('Data/2022 VNL/JPA VS FRA.xlsx', 'JAP', 3)
df_dict['df_FRA_2'] = get_data('Data/2022 VNL/JPA VS FRA.xlsx', 'FRA', 3)

df_dict['df_USA_1'] = get_data('Data/2022 VNL/USA VS BRA.xlsx', 'USA', 3)
df_dict['df_BRA_2'] = get_data('Data/2022 VNL/USA VS BRA.xlsx', 'BRA', 3)

df_dict['df_USA_2'] = get_data('Data/2022 VNL/USA VS FRA.xlsx', 'USA', 5)
df_dict['df_FRA_3'] = get_data('Data/2022 VNL/USA VS FRA.xlsx', 'FRA', 5)

df_dict['df_USA_3'] = get_data('Data/2022 VNL/USA VS POL.xlsx', 'USA', 3)
df_dict['df_BRA_2'] = get_data('Data/2022 VNL/USA VS POL.xlsx', 'POL', 3)

In [189]:
for i, df in df_dict.items():
    print(len(df), end=' ')

454 397 285 306 425 486 365 344 297 297 376 311 445 435 298 

In [191]:
df = pd.concat([df for i, df in df_dict.items()])

df = df.sample(frac=1).reset_index(drop=True)   # frac: the persentage of dataset to be remained
print(len(df))
df.head()

5521


,Team,No,Space,Action,Erros,Attempts,Score
0,ITA,7,8,8,1.0,0.0,0
1,USA,2,10,6,0.0,0.0,0
2,POL,14,4,5,0.0,0.0,0
3,NED,12,13,3,0.0,0.0,0
4,POL,21,7,8,0.0,0.0,0


In [192]:
df.to_csv('df.csv', index=False)